## Simple Translation

In [ ]:
# To follow along, run this cell first.

options(warn = -1)
suppressMessages(library(neotoma2))
suppressMessages(library(sf))
suppressMessages(library(geojsonsf))
suppressMessages(library(dplyr))
suppressMessages(library(ggplot2))
suppressMessages(library(leaflet))
options(dplyr.summarise.inform = FALSE)

cz_dl <- readRDS('data/czDownload.RDS')
allSamp <- samples(cz_dl)

In [ ]:
allSamp <- allSamp %>% 
  mutate(variablename = replace(variablename, 
                                stringr::str_detect(variablename, "Plantago.*"), 
                                "Plantago"))
DT::datatable(data = head(allSamp, n = 5), rownames = FALSE)

In [ ]:
taxaSites <- allSamp %>%
  group_by(variablename, siteid) %>% 
  group_by(variablename) %>% 
  summarise(sites = length(unique(siteid)))
DT::datatable(data = head(taxaSites, n = 15), rownames = FALSE)

In [ ]:
taxaSamples <- allSamp %>%
  group_by(variablename) %>% 
  summarise(samples = n())
DT::datatable(data = head(taxaSamples, n = 8), rownames = FALSE)

In [ ]:
taxaplots <- taxaSites %>% inner_join(taxaSamples, by = "variablename")
DT::datatable(data = head(taxaplots, n = 20), rownames = FALSE)

In [ ]:
ggplot(data = taxaplots, aes(x = sites, y = samples)) +
  geom_point() +
  stat_smooth(method = 'glm', 
              method.args = list(family = 'poisson')) +
  xlab("Number of Sites") +
  ylab("Number of Samples") +
  theme_bw()

In [ ]:
translation <- readr::read_csv("data/taxontable.csv", show_col_types = FALSE)
DT::datatable(data = head(translation, n = 8), rownames = FALSE)

In [ ]:
allSamp <- samples(cz_dl)

allSamp <- allSamp %>%
  inner_join(translation, by = c("variablename" = "variablename")) %>% 
  select(!c("variablename", "sites", "samples")) %>% 
  group_by(siteid, sitename, replacement,
           sampleid, units, age,
           agetype, depth, datasetid,
           long, lat) %>%
  summarise(value = sum(value))

In [ ]:
DT::datatable(head(allSamp, n = 5), rownames = FALSE)